## Overview
Geospatial query capabilities in Snowflake are built upon a combination of data types and specialized query functions that can be used to parse, construct, and run calculations over geospatial objects. This guide will introduce you to the `GEOMETRY` data type, help you understand geospatial formats supported by Snowflake and walk you through the use of a variety of functions on sample geospatial data sets. 

### What You’ll Learn
* How to acquire geospatial data from the Snowflake Marketplace
* How to load geospatial data from a Stage
* How to interpret the `GEOMETRY` data type and how it differs from the `GEOGRAPHY`
* How to understand the different formats that `GEOMETRY` can be expressed in
* How to do spatial analysis using the `GEOMETRY` and `GEOGRAPHY` data types
* How to use Python UDFs for reading Shapefiles and creating custom functions
* How to visualise geospatial data using Streamlit

### What You’ll Build
A sample use case that involves energy grids and LTE cell towers in the Netherlands You will answer the following questions:
* What is the length of all energy grids in each municipality in the Netherlands?
* What cell towers lack electricity cables nearby?

## Acquire Marketplace Data and Analytics Toolbox

The first step in the guide is to acquire geospatial data sets that you can freely use to explore the basics of Snowflake's geospatial functionality.  The best place to acquire this data is the Snowflake Marketplace!  
* Navigate to the `Marketplace` screen using the menu on the left side of the window
* Search for `OpenCelliD` in the search bar
* Find and click the` OpenCelliD - Open Database of Cell Towers` tile or just use [this](https://app.snowflake.com/marketplace/listing/GZSVZ8ON6J/dataconsulting-pl-opencellid-open-database-of-cell-towers) link
* Once in the listing, click the big blue `Get` button
* On the `Get Data` screen, change the name of the database from the default to `OPENCELLID`, as this name is shorter, and all future instructions will assume this name for the database.

Similarly to the above dataset, acquire [SedonaSnow](https://app.snowflake.com/marketplace/listing/GZTYZF0RTY3/wherobots-sedonasnow) application which extends Snowflake core geo features with more than 100 spatial functions. Navigate to the `Marketplace` screen using the menu on the left side of the window and find the `SedonaSnow`. Keep the the database name `SEDONASNOW` and optionally add more roles that can access the database.

Congratulations! You have just acquired all the listings you need for this lab.

## Setup your Account

Create a new database and schema where you will store datasets in the `GEOMETRY` data type. Run th following SQL:

In [ ]:
CREATE DATABASE IF NOT EXISTS ADVANCED_ANALYTICS;
CREATE SCHEMA IF NOT EXISTS ADVANCED_ANALYTICS.GEOMETRY;
USE SCHEMA ADVANCED_ANALYTICS.GEOMETRY;

## Load Data from External Storage

You already understand how to get data from Marketplace, let's try another way of getting data, namely, getting it from the external S3 storage. While you loading data you will learn formats supported by geospatial data types.

For this quickstart we have prepared a dataset with energy grid infrastructure (cable lines) in the Netherlands. It is stored in the CSV format in the public S3 bucket. To import this data, create an external stage using the following SQL command:



In [ ]:
CREATE OR REPLACE STAGE ADVANCED_ANALYTICS.GEOMETRY.GEOSTAGE
  URL = 's3://sfquickstarts/vhol_spatial_analysis_geometry_geography/';

Look inside of newly created stage:

In [ ]:
list @ADVANCED_ANALYTICS.GEOMETRY.GEOSTAGE

Now you will create a new table using the file from that stage. Run the following queries to create a new file format and a new table using the dataset stored in the Stage:

In [ ]:
// Create file format
CREATE OR REPLACE FILE FORMAT geocsv TYPE = CSV SKIP_HEADER = 1 FIELD_OPTIONALLY_ENCLOSED_BY = '"';

CREATE OR REPLACE TABLE ADVANCED_ANALYTICS.GEOMETRY.NL_CABLES_STATIONS AS 
SELECT to_geometry($1) AS geometry, 
       $2 AS id, 
       $3 AS type 
FROM @geostage/nl_stations_cables.csv (file_format => 'geocsv');

Look at the description of the table you just created by running the following queries: 

In [ ]:
DESC TABLE ADVANCED_ANALYTICS.GEOMETRY.NL_CABLES_STATIONS;

The [desc or describe](https://docs.snowflake.com/en/sql-reference/sql/desc.html) command shows you the definition of the view, including the columns, their data type, and other relevant details. Notice the `geometry` column is defined as `GEOMETRY` type. 

Snowflake supports 3 primary geospatial formats and 2 additional variations on those formats. They are:

* **GeoJSON**: a JSON-based standard for representing geospatial data
* **WKT & EWKT**: a "Well Known Text" string format for representing geospatial data and the "Extended" variation of that format
* **WKB & EWKB:** a "Well Known Binary" format for representing geospatial data in binary and the "Extended" variation of that format

These formats are supported for ingestion (files containing those formats can be loaded into a `GEOMETRY` typed column), query result display, and data unloading to new files. You don't need to worry about how Snowflake stores the data under the covers but rather how the data is displayed to you or unloaded to files through the value of session variables called `GEOMETRY_OUTPUT_FORMAT`.

Run the query below to make sure the current format is GeoJSON:

In [ ]:
ALTER SESSION SET geometry_output_format = 'GEOJSON';

The [alter session](https://docs.snowflake.com/en/sql-reference/sql/alter-session.html) command lets you set a parameter for your current user session, which in this case is  `GEOMETRY_OUTPUT_FORMAT`. The default value for those parameters is `'GEOJSON'`, so normally you wouldn't have to run this command if you want that format, but this guide wants to be certain the next queries are run with the `'GEOJSON'` output.

Now run the following query against the `nl_cables_stations` table to see energy grids in the Netherlands.

In [ ]:
SELECT geometry
FROM nl_cables_stations
LIMIT 5;

In the result set, notice the `GEOMETRY` column and how it displays a JSON representation of spatial objects. It should look similar to this:

```
{"coordinates": [[[1.852040750000000e+05, 3.410349640000000e+05], 
[1.852044840000000e+05,3.410359860000000e+05]], 
[[1.852390240000000e+05,3.411219340000000e+05], 
... ,
[1.852800600000000e+05,3.412219960000000e+05]]   ], 
"type": "MultiLineString" }
```

Unlike `GEOGRAPHY`, which treats all points as longitude and latitude on a spherical earth, `GEOMETRY` considers the Earth as a flat surface. More information about Snowflake's specification can be found [here](https://docs.snowflake.com/en/sql-reference/data-types-geospatial.html).
In this example it uses scientific notation and the numbers are much larger than latitude and longitude boundaries [-180; 180].

Now look at the same query but in a different format. Run the following query:

In [ ]:
ALTER SESSION SET geometry_output_format = 'EWKT';

Run the previous `SELECT` query again and when done, examine the output in the `GEOMETRY` column.

In [ ]:
SELECT geometry
FROM nl_cables_stations
LIMIT 5;

EWKT looks different from GeoJSON, and is arguably more readable. Here you can more clearly see the [geospatial object types](https://docs.snowflake.com/en/sql-reference/data-types-geospatial.html#geospatial-object-types). EWKT also shows the spatial reference identifier and in our example, you have a dataset in [Amersfoort / RD New](https://epsg.io/28992) spatial reference system, that is why the displayed SRID is 28992.

Lastly, look at the WKB output. Run the following query:

In [ ]:
ALTER SESSION SET geometry_output_format = 'WKB';

Run the query again:

In [ ]:
SELECT geometry
FROM nl_cables_stations 
LIMIT 5;

Now that you have a basic understanding of how the `GEOMETRY` data type works and what a geospatial representation of data looks like in various output formats, it's time to walk through a scenario that requires you to use constructors to load data.  You will do it while trying one more way of getting data, namely, from the Shapefile file stored in the external stage. 

One of the files in the external stage contains the polygons of administrative boundaries in the Netherlands. The data is stored in [Shapefile format](https://en.wikipedia.org/wiki/Shapefile) which is not yet supported by Snowflake. But you can load this file using Python UDF and [Dynamic File Access feature](https://docs.snowflake.com/developer-guide/udf/python/udf-python-examples#label-udf-python-read-files). You will also use some packages available in the Snowflake Anaconda channel.

Run the following query that creates a UDF to read shapfiles:

In [ ]:
CREATE OR REPLACE FUNCTION ADVANCED_ANALYTICS.GEOMETRY.PY_LOAD_GEODATA(PATH_TO_FILE string, filename string)
RETURNS TABLE (wkt varchar, properties object)
LANGUAGE PYTHON
RUNTIME_VERSION = 3.8
PACKAGES = ('fiona', 'shapely', 'snowflake-snowpark-python')
HANDLER = 'GeoFileReader'
AS $$
from shapely.geometry import shape
from snowflake.snowpark.files import SnowflakeFile
from fiona.io import ZipMemoryFile
class GeoFileReader:        
    def process(self, PATH_TO_FILE: str, filename: str):
    	with SnowflakeFile.open(PATH_TO_FILE, 'rb') as f:
    		with ZipMemoryFile(f) as zip:
    			with zip.open(filename) as collection:
    				for record in collection:
    					yield (shape(record['geometry']).wkt, dict(record['properties']))
$$;

This UDF reads a Shapefile and returns its content as a table. Under the hood it uses geospatial libraries `fiona` and `shapely`.
Run the following query to see the content of the uploaded shapefile.

In [ ]:
ALTER SESSION SET geometry_output_format = 'EWKT';

SELECT to_geometry(wkt) AS geometry,
       properties:NAME_1::string AS province_name,
       properties:NAME_2::string AS municipality_name
FROM table(py_load_geodata(build_scoped_file_url(@ADVANCED_ANALYTICS.GEOMETRY.GEOSTAGE, 'nl_areas.zip'), 'nl_areas.shp'));

This query fails with the error *Geometry validation failed*. The constructor function determines if the shape is valid according to the [Open Geospatial Consortium’s Simple Feature Access / Common Architecture](https://www.ogc.org/standards/sfa) standard. If the shape is invalid, the function reports an error and does not create the GEOMETRY object. That is what happened in our example.

To fix this you can allow the ingestion of invalid shapes by setting the corresponding parameter to True. Let's run the SELECT statement again, but update the query to see how many shapes are invalid. Run the following query:

In [ ]:
SELECT to_geometry(s => wkt, allowInvalid => True) AS geometry,
       st_isvalid(geometry) AS is_valid,
       properties:NAME_1::string AS province_name,
       properties:NAME_2::string AS municipality_name
FROM table(py_load_geodata(build_scoped_file_url(@ADVANCED_ANALYTICS.GEOMETRY.GEOSTAGE, 'nl_areas.zip'), 'nl_areas.shp'))
ORDER BY is_valid ASC;

This query completed without error and now you see that the shape of the province Zeeland is invalid. Let's repair it by applying the [ST_MakeValid](https://sedona.apache.org/1.5.1/api/snowflake/vector-data/Function/#st_makevalid) function from SedonaSnow Native app:

In [ ]:
SELECT SEDONASNOW.SEDONA.st_MakeValid(to_geometry(s => wkt, allowInvalid => True)) AS geometry,
       st_isvalid(geometry) AS is_valid,
       (CASE WHEN properties:TYPE_1::string IS NULL THEN 'Municipality' ELSE 'Province' END) AS type,
       properties:NAME_1::string AS province_name,
       properties:NAME_2::string AS municipality_name
FROM table(py_load_geodata(build_scoped_file_url(@ADVANCED_ANALYTICS.GEOMETRY.GEOSTAGE, 'nl_areas.zip'), 'nl_areas.shp'))
ORDER BY is_valid ASC;

Now all shapes are valid and the data is ready to be ingested. One additional thing you should do is to set SRID, since otherwise it will be set to 0. This dataset is in the reference system [WGS 72 / UTM zone 31N](https://epsg.io/32231), so it makes sense to add the SRID=32231 to the constructor function.

Run the following query:

In [ ]:
CREATE OR REPLACE TABLE ADVANCED_ANALYTICS.GEOMETRY.NL_ADMINISTRATIVE_AREAS AS
SELECT ST_SETSRID(SEDONASNOW.SEDONA.ST_MakeValid(to_geometry(s => wkt, srid => 32231, allowInvalid => True)), 32231) AS geometry,
       st_isvalid(geometry) AS is_valid,
       (CASE WHEN properties:TYPE_1::string IS NULL THEN 'Municipality' ELSE 'Province' END) AS type,
       properties:NAME_1::string AS province_name,
       properties:NAME_2::string AS municipality_name
FROM table(py_load_geodata(build_scoped_file_url(@ADVANCED_ANALYTICS.GEOMETRY.GEOSTAGE, 'nl_areas.zip'), 'nl_areas.shp'))
ORDER BY is_valid ASC;

Excellent! Now that all the datasets are successfully loaded, let's proceed to the next exciting step: the analysis.

## Spatial Analysis

To showcase the capabilities of the GEOMETRY data type, you will explore several use cases. In these scenarios, you'll assume you are an analyst working for an energy utilities company responsible for maintaining electrical grids.

### What is the length of the electricity cables?
In the first use case you will calculate the length of electrical cables your organization is responsible for in each administrative area within the Netherlands. You'll be utilizing two datasets: with power infrastructure of the Netherlands and the borders of Dutch administrative areas. First, let's check the sample of each dataset.

Run the following query to see the content of `nl_cables_stations` table:

In [ ]:
SELECT geometry, type
FROM ADVANCED_ANALYTICS.GEOMETRY.NL_CABLES_STATIONS
LIMIT 5;

The spatial data is stored using the `GEOMETRY` data type and employs the Dutch mapping system, `Amersfoort / RD New` (SRID = 28992). 

To view the contents of the table containing the boundaries of the administrative areas in the Netherlands, execute the following query:

In [ ]:
SELECT *
FROM ADVANCED_ANALYTICS.GEOMETRY.NL_ADMINISTRATIVE_AREAS
LIMIT 5;

In order to compute the length of all cables per administrative area, it's essential that both datasets adhere to the same mapping system. You have two options: either project `nl_administrative_areas` to SRID 28992, or project `nl_cables_stations` to SRID 32231. For this exercise, let's choose the first option. Run the following query:

In [ ]:
SELECT t1.province_name,
       sum(st_length(t2.geometry)) AS cables_length
FROM ADVANCED_ANALYTICS.GEOMETRY.NL_ADMINISTRATIVE_AREAS AS t1,
     ADVANCED_ANALYTICS.GEOMETRY.NL_CABLES_STATIONS AS t2
WHERE st_intersects(st_transform(t1.geometry, 28992), t2.geometry)
  AND t1.type = 'Province'
GROUP BY 1
ORDER BY 2 DESC;

You have five areas densely covered by electricity cables, those are the ones that your company is responsible for. For your first analysis, you will focus on these areas.

### What cell towers lack electricity cables nearby

In many areas, especially rural or remote ones, cell towers might be located far from electricity grids. This can pose a challenge in providing a reliable power supply to these towers. They often rely on diesel generators, which can be expensive to operate and maintain and have environmental implications. Furthermore, power outages can lead to disruptions in mobile connectivity, impacting individuals, businesses, and emergency services.

Our analysis aims to identify mobile cell towers that are not near an existing electricity grid. This information could be used to prioritize areas for grid expansion, to improve the efficiency of renewable energy source installations (like solar panels or wind turbines), or to consider alternative energy solutions.

For this and the next examples let's use `GEOGRAPHY` data type as it can be easily visualized using CARTO. As a first step, let's create `GEOGRAPHY` equivalents for the energy grids and boundaries tables. For that you need to project the `geometry` column in each of the tables into the mapping system WGS 84 (SRID=4326) and then convert to `GEOGRAPHY` data type. Run the following queries that create new tables and enable search optimization for each of them in order to increase the performance of spatial operations. 

In [ ]:
// Creating a table with GEOGRAPHY for nl_administrative_areas
CREATE SCHEMA IF NOT EXISTS ADVANCED_ANALYTICS.GEOGRAPHY;

CREATE OR REPLACE TABLE ADVANCED_ANALYTICS.GEOGRAPHY.NL_ADMINISTRATIVE_AREAS AS
SELECT to_geography(st_transform(geometry, 4326)) AS geom,
       type,
       province_name,
       municipality_name
FROM ADVANCED_ANALYTICS.GEOMETRY.nl_administrative_areas
ORDER BY st_geohash(geom);

// Creating a table with GEOGRAPHY for NL_CABLES_STATIONS
CREATE OR REPLACE TABLE ADVANCED_ANALYTICS.GEOGRAPHY.NL_CABLES_STATIONS AS
SELECT to_geography(st_transform(geometry, 4326)) AS geom,
       id,
       type
FROM ADVANCED_ANALYTICS.GEOMETRY.NL_CABLES_STATIONS
ORDER BY st_geohash(geom);

Now you will create a table with locations of cell towers stored as GEOGRAPHY, just like for the previous two tables. Run the following query:

In [ ]:
CREATE OR REPLACE TABLE ADVANCED_ANALYTICS.GEOGRAPHY.NL_LTE AS
SELECT DISTINCT st_point(lon, lat) AS geom,
                cell_range
FROM OPENCELLID.PUBLIC.RAW_CELL_TOWERS t1
WHERE mcc = '204' -- 204 is the mobile country code in the Netherlands
  AND radio='LTE'

Finally, you will find all cell towers that don't have an energy line within a 2-kilometer radius. For each cell tower you'll calculate the distance to the nearest electricity cable. You will use Streamlit library `pydeck` to visualise municipalities and locations of cell towers. 

As a  preparation step you need to import pydeck library that you will use in this Lab. Navigate to the `Packages` drop-down  in the upper right of the Notebook and search for `pydeck`. Click on `pydeck` to add it to the Python packages. Then run the following Python code:

In [ ]:
import streamlit as st
import pandas as pd
import pydeck as pdk
import json
from snowflake.snowpark.context import get_active_session

session = get_active_session()

def get_celltowers() -> pd.DataFrame:
    return session.sql(f"""
    SELECT province_name,
    cells.geom
    FROM ADVANCED_ANALYTICS.GEOGRAPHY.nl_lte cells
    LEFT JOIN ADVANCED_ANALYTICS.GEOGRAPHY.NL_CABLES_STATIONS cables
    ON st_dwithin(cells.geom, cables.geom, 2000)
    JOIN ADVANCED_ANALYTICS.GEOGRAPHY.NL_ADMINISTRATIVE_AREAS areas 
    ON st_contains(areas.geom, cells.geom)
    WHERE areas.type = 'Municipality'
    AND areas.province_name in ('Noord-Brabant', 'Overijssel', 'Limburg', 'Groningen', 'Drenthe')
    AND cables.geom IS NULL; """).to_pandas()

def get_boundaries() -> pd.DataFrame:
    return session.sql(f"""
        SELECT st_simplify(GEOM, 10) as geom, municipality_name
        FROM ADVANCED_ANALYTICS.GEOGRAPHY.NL_ADMINISTRATIVE_AREAS
        WHERE type = 'Municipality';
    """).to_pandas()


boundaries = get_boundaries()
boundaries["coordinates"] = boundaries["GEOM"].apply(lambda row: json.loads(row)["coordinates"][0])

celltowers = get_celltowers()
celltowers["lon"] = celltowers["GEOM"].apply(lambda row: json.loads(row)["coordinates"][0])
celltowers["lat"] = celltowers["GEOM"].apply(lambda row: json.loads(row)["coordinates"][1])

layer_celltowers = pdk.Layer(
            "ScatterplotLayer",
            celltowers,
            get_position=["lon", "lat"],
            id="celltowers",
            stroked=True,
            filled=True,
            extruded=False,
            wireframe=True,
            get_fill_color=[233, 43, 65],
            get_line_color=[233, 43, 65],
            get_radius=300,
            auto_highlight=True,
            pickable=False,
        )

layer_boundaries = pdk.Layer(
    "PolygonLayer",
    data=boundaries,
    id="province-layer",
    get_polygon="coordinates",
    extruded=False,
    opacity=0.9,
    wireframe=True,
    pickable=True,
    stroked=True,
    filled=True,
    line_width_min_pixels=1,
    get_line_color=[17, 86, 127],       # Red color for the border
    get_fill_color=[43, 181, 233, 30],  # Blue fill with transparency
    coverage=1
)


st.pydeck_chart(pdk.Deck(
    map_style=None,
    initial_view_state=pdk.ViewState(
        latitude=51.97954426323304,
        longitude=5.626041932127842, 
        # pitch=45, 
        zoom=8),
    tooltip={
            'html': '<b>Province name:</b> {MUNICIPALITY_NAME}',
             'style': {
                 'color': 'white'
                 }
            },
    layers=[layer_boundaries, layer_celltowers],
))


Another way to visualise geospatial data is using open-source geo analytics tool QGIS. Do the following steps:
* Install the latest [Long Term Version of QGIS](https://qgis.org/download/)
* Install Snowflake conector. Go to `Plugins` > `All`, search for `Snowflake Connector for QGIS` and click `Install Plugin`.
* Go to `Layer` > `Data Source Manager` and create a new connection to Snowflake. Call it `SNOWFLAKE` (all letters capital). [Check](https://github.com/snowflakedb/qgis-snowflake-connector?tab=readme-ov-file#getting-started) the documentation to learn mor on how to create new coonection
* [Download](https://sfquickstarts.s3.us-west-1.amazonaws.com/vhol_spatial_analysis_geometry_geography/energy_grids_nl.qgz) a QGIS project that we created for you and open it in QGIS.
* If previous steps done correctly, you should be able to see the following layers in QGIS
    * `ENERGY_GRIDS` (LINESTRING and MULTILINESTRING) - energy frids for Noord-Brabant, Overijssel, Limburg, Groningen, and Drenthe.
    * `CELL_TOWERS_WITHOUT_CABLES` - cell towers in the regions above that don't have energy grids within radius of 2km.
    * `Municipalities` (POLYGON and MULTIPOLYGON) - Boundaries of Dutch municipalities.


## Conclusion

In this guide, you acquired geospatial data from the Snowflake Marketplace, explored how the `GEOMETRY` data type works and how it differs from `GEOGRAPHY`. You converted one data type into another and queried geospatial data using parser, constructor, transformation, and used geospatial joins. You then saw how geospatial objects could be visualized using CARTO.

You are now ready to explore the larger world of Snowflake geospatial support and geospatial functions.

### What we've covered
* How to acquire a shared database from the Snowflake Marketplace and from External and internal storages.
* The GEOMETRY data type, its formats GeoJSON, WKT, EWKT, WKB, and EWKB, and how to switch between them.
* How to use constructors like TO_GEOMETRY, ST_MAKELINE.
* How to reproject between SRIDs using ST_TRANSFORM.
* How to perform relational calculations like ST_DWITHIN and ST_INTERSECTS.
* How to perform measurement calculations like ST_LENGTH.
* How to use Python UDFs for reading Shapefiles and creating custom functions.
* How to visualise geospatial data using Streamlit and QGIS